In [1]:
%pip install torch torchtext transformers sentencepiece pandas tqdm datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

c:\Users\Outis Lupin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#load dataset
db = load_dataset("QuyenAnhDE/Diseases_Symptoms")

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
db

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [5]:
updated_db = [{'Name':item['Name'], 'Symptoms':item['Symptoms']} for item in db['train']]

In [6]:
df = pd.DataFrame(updated_db)

In [7]:
df.head(5)

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [8]:
#extracting the symptoms feature/column
df['Symptoms'] = df['Symptoms'].apply(lambda x: ', '.join(x.split(', ')))

In [9]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

In [10]:
#setting up the device with cuda or cpu
if torch.cuda.is_available():
    device = torch.device('cuda')
    
else:
    device = torch.device('cpu')

In [11]:
device

device(type='cpu')

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)
#force_download=True

c:\Users\Outis Lupin\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [14]:
BATCH_SIZE = 8

In [15]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


# Dataset Preparation

In [16]:
class LanguageDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.labels = df.columns 
        self.data = df.to_dict(orient='records')
        self.tokenizer = tokenizer
        x = self.fittest_max_length(df)
        self.max_length = x
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = self.data[idx][self.labels[0]]
        y = self.data[idx][self.labels[1]]
        text = f"{x} | {y}"
        tokens = self.tokenizer.encode_plus(text, return_tensors = 'pt', max_length=128, padding='max_length', truncation=True)
        return tokens
    
    def fittest_max_length(self, df):
        max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
        x = 2
        while x < max_length: x = x * 2
        return x

In [17]:
data_sample = LanguageDataset(df, tokenizer)

In [18]:
data_sample

# Data Preprocessing 

In [19]:
train_size = int(0.8 * len(data_sample))
val_size = len(data_sample) - train_size

train_data, val_data = random_split(data_sample, [train_size, val_size])

In [20]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

In [21]:
num_epochs = 10

In [22]:
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0

In [23]:
# Set the learning rate and loss function
## CrossEntropyLoss measures how close answers to the truth.


criterion = nn.CrossEntropyLoss(ignore_index= tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [24]:
results = pd.DataFrame(columns=['epoch','transformer','batch_size', 'gpu', 'training_loss', 'validation_loss', 'epoch_duration_sec'])

In [25]:
# The training loop
for epoch in range(num_epochs):
    start_time = time.time()  # Start the timer for the epoch

    # Training
    ## This line tells the model we're in 'learning mode'
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    # Validation
    ## This line below tells the model to 'stop learning'
    model.eval()
    epoch_validation_loss = 0
    total_loss = 0
    valid_iterator = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({'Validation Loss': loss.item()})
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(val_loader)

    end_time = time.time()  # End the timer for the epoch
    epoch_duration_sec = end_time - start_time  # Calculate the duration in seconds

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}  # Add epoch_duration to the dataframe

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(val_loader)}")

Training epoch 1/8 Batch Size: 8, Transformer:distilgpt2: 100%|██████████| 40/40 [03:17<00:00,  4.94s/it, Training Loss=0.581]
validating epoch 1/8: 100%|██████████| 10/10 [00:14<00:00,  1.50s/it, Validation Loss=<built-in method item of Tensor object at 0x0000029CA10E3F20>]


Epoch: 1, Validation Loss: 0.693556547164917


Training epoch 2/8 Batch Size: 8, Transformer:distilgpt2: 100%|██████████| 40/40 [03:07<00:00,  4.70s/it, Training Loss=0.405]
validating epoch 2/8: 100%|██████████| 10/10 [00:14<00:00,  1.47s/it, Validation Loss=<built-in method item of Tensor object at 0x0000029CA1125400>]


Epoch: 2, Validation Loss: 0.664656937122345


Training epoch 3/8 Batch Size: 8, Transformer:distilgpt2: 100%|██████████| 40/40 [03:08<00:00,  4.70s/it, Training Loss=0.484]
validating epoch 3/8: 100%|██████████| 10/10 [00:14<00:00,  1.45s/it, Validation Loss=<built-in method item of Tensor object at 0x0000029CA1131E00>]


...

Epoch: 6, Validation Loss: 0.8033286929130554


Training epoch 7/8 Batch Size: 8, Transformer:distilgpt2: 100%|██████████| 40/40 [03:03<00:00,  4.59s/it, Training Loss=0.126]
validating epoch 7/8: 100%|██████████| 10/10 [00:14<00:00,  1.45s/it, Validation Loss=<built-in method item of Tensor object at 0x0000029CA1131450>]


Epoch: 7, Validation Loss: 0.8658673167228699


Training epoch 8/8 Batch Size: 8, Transformer:distilgpt2: 100%|██████████| 40/40 [03:03<00:00,  4.59s/it, Training Loss=0.145] 
validating epoch 8/8: 100%|██████████| 10/10 [00:14<00:00,  1.44s/it, Validation Loss=<built-in method item of Tensor object at 0x0000029CA1133750>]

Epoch: 8, Validation Loss: 0.901246190071106


#### We have faced overfittig

# Model Performing(testing)

In [34]:
input_str = "Depression"

In [35]:
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

In [36]:
input_ids

tensor([[12156,  2234]])

In [37]:
output = model.generate(
    input_ids,
    max_length=20,
    num_return_sequences = 1,
    do_sample = True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [38]:
output

tensor([[12156,  2234,   930,  2275,  3438,  1292,  2356,    11, 24924,   803,
            11,  2458,   287, 35711,  8650, 50256]])

In [39]:
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

In [42]:
decoded_output

Depression | Abdominal pain, bloating, changes in bowel movements


# Saving the model

In [41]:
torch.save(model, 'DiseaseSLM')